In [20]:
import os
import pandas as pd
from datetime import datetime
import glob

In [21]:
base_dir = "bike_data"  # Root directory with all month folders
output_file = "citibike_hourly_summary_2020_2023.csv"

In [22]:
def extract_month(folder_name):
    return folder_name.split("-")[0] 

In [26]:
def process_csv(file_path, expected_month):
    chunks = pd.read_csv(file_path, chunksize=100_000, low_memory=False, parse_dates=['started_at', 'ended_at'])
    results = []

    for chunk in chunks:
        # Ensure datetime conversion
        chunk = chunk.dropna(subset=['started_at', 'ended_at'])
        chunk['started_at'] = pd.to_datetime(chunk['started_at'], errors='coerce')
        chunk['ended_at'] = pd.to_datetime(chunk['ended_at'], errors='coerce')
        chunk = chunk.dropna(subset=['started_at', 'ended_at'])

        # Filter to rides that start and end in the correct month
        chunk = chunk[
            (chunk['started_at'].dt.strftime("%Y%m") == expected_month) &
            (chunk['ended_at'].dt.strftime("%Y%m") == expected_month)
        ]

        if chunk.empty:
            continue

        # Calculate ride duration and round to hour
        chunk['ride_duration_min'] = (chunk['ended_at'] - chunk['started_at']).dt.total_seconds() / 60
        chunk['hour'] = chunk['started_at'].dt.floor('h')

        # Group by hour
        hourly = chunk.groupby('hour').agg(
            ride_count=('ride_id', 'count'),
            avg_ride_duration_min=('ride_duration_min', 'mean')
        ).reset_index()

        results.append(hourly)

    return pd.concat(results, ignore_index=True) if results else pd.DataFrame(columns=['hour', 'ride_count', 'avg_ride_duration_min'])

In [27]:
all_data = []

In [28]:
for folder_name in sorted(os.listdir(base_dir)):
    folder_path = os.path.join(base_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue

    expected_month = extract_month(folder_name)
    print(f"📁 Processing month: {expected_month} ({folder_name})")

    # Get all CSVs inside this folder matching *_citibike-tripdata_*.csv
    csv_files = sorted(glob.glob(os.path.join(folder_path, f"{expected_month}-citibike-tripdata_*.csv")))

    for file_path in csv_files:
        print(f"  📄 File: {os.path.basename(file_path)}")
        try:
            df = process_csv(file_path, expected_month)
            if not df.empty:
                all_data.append(df)
        except Exception as e:
            print(f"    ❌ Error processing {file_path}: {e}")


📁 Processing month: 202001 (202001-citibike-tripdata)
  📄 File: 202001-citibike-tripdata_1.csv
  📄 File: 202001-citibike-tripdata_2.csv
📁 Processing month: 202002 (202002-citibike-tripdata)
  📄 File: 202002-citibike-tripdata_1.csv
  📄 File: 202002-citibike-tripdata_2.csv
📁 Processing month: 202003 (202003-citibike-tripdata)
  📄 File: 202003-citibike-tripdata_1.csv
  📄 File: 202003-citibike-tripdata_2.csv
📁 Processing month: 202004 (202004-citibike-tripdata)
  📄 File: 202004-citibike-tripdata_1.csv
📁 Processing month: 202005 (202005-citibike-tripdata)
  📄 File: 202005-citibike-tripdata_1.csv
  📄 File: 202005-citibike-tripdata_2.csv
📁 Processing month: 202006 (202006-citibike-tripdata)
  📄 File: 202006-citibike-tripdata_1.csv
  📄 File: 202006-citibike-tripdata_2.csv
📁 Processing month: 202007 (202007-citibike-tripdata)
  📄 File: 202007-citibike-tripdata_1.csv
  📄 File: 202007-citibike-tripdata_2.csv
  📄 File: 202007-citibike-tripdata_3.csv
📁 Processing month: 202008 (202008-citibike-trip

/var/folders/96/3rn3swsn3m10f02c6cqx2fq00000gn/T/ipykernel_24059/1737364568.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunks:


  📄 File: 202108-citibike-tripdata_2.csv
  📄 File: 202108-citibike-tripdata_3.csv
  📄 File: 202108-citibike-tripdata_4.csv
📁 Processing month: 202109 (202109-citibike-tripdata)
  📄 File: 202109-citibike-tripdata_1.csv
  📄 File: 202109-citibike-tripdata_2.csv
  📄 File: 202109-citibike-tripdata_3.csv
  📄 File: 202109-citibike-tripdata_4.csv
📁 Processing month: 202110 (202110-citibike-tripdata)
  📄 File: 202110-citibike-tripdata_1.csv
  📄 File: 202110-citibike-tripdata_2.csv
  📄 File: 202110-citibike-tripdata_3.csv
  📄 File: 202110-citibike-tripdata_4.csv
📁 Processing month: 202111 (202111-citibike-tripdata)
  📄 File: 202111-citibike-tripdata_1.csv
  📄 File: 202111-citibike-tripdata_2.csv
  📄 File: 202111-citibike-tripdata_3.csv
📁 Processing month: 202112 (202112-citibike-tripdata)
  📄 File: 202112-citibike-tripdata_1.csv
  📄 File: 202112-citibike-tripdata_2.csv
📁 Processing month: 202201 (202201-citibike-tripdata)
  📄 File: 202201-citibike-tripdata_1.csv
  📄 File: 202201-citibike-tripdat

In [29]:
if all_data:
    combined = pd.concat(all_data, ignore_index=True)
    # Aggregate again in case same hour appears in multiple files
    final = combined.groupby('hour').agg(
        ride_count=('ride_count', 'sum'),
        avg_ride_duration_min=('avg_ride_duration_min', 'mean')
    ).reset_index()

    final.to_csv(output_file, index=False)
    print(f"\n✅ Done! Saved to: {output_file}")
else:
    print("⚠️ No data was aggregated.")


✅ Done! Saved to: citibike_hourly_summary_2020_2023.csv


In [35]:
citibike_df = pd.read_csv("citibike_hourly_summary_2020_2023.csv", parse_dates=["time"])
weather_df = pd.read_csv("weather_data/new_york_weather_2020_2023.csv", parse_dates=["time"])

merged_df = pd.merge(
    citibike_df,
    weather_df,
    on="time",
    how="inner"
)

merged_df.to_csv("citibike_weather_merged_2020_2023.csv", index=False)
print("done")

done


In [38]:
df1 = pd.read_csv("citibike_weather_merged_2020_2023.csv", parse_dates=["time"])
df2 = pd.read_csv("citibike_hourly_with_weather_2023_2025.csv", parse_dates=["time"])

combined_df = pd.concat([df1, df2], ignore_index = True)

combined_df.sort_values(by="time", inplace=True)

combined_df.to_csv("citibike_weather_merged_2020_to_2025.csv", index=False)
print("done")

done
